In [ ]:
import numpy as np
import pandas as pd
import seaborn as sns
import datetime as datetime
import random
import matplotlib.pyplot as plt
%matplotlib inline

#preprocessing 
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer

#models
import xgboost as xgb

In [ ]:
#these vars are determined from the xgb f score script
df_n_prep = pd.read_csv('train_numeric.csv',usecols=['Id',
             'L3_S32_F3850','L0_S0_F0','L0_S12_F330','L1_S24_F1512','L3_S38_F3952','L3_S33_F3855',
             'L1_S24_F1514','L3_S33_F3859','L1_S24_F1118','L1_S25_F1855','L1_S24_F1723','L3_S38_F3960',
             'L0_S0_F18','L3_S30_F3494','L0_S0_F8','L0_S0_F2','L2_S27_F3129','L1_S24_F1518','L3_S30_F3749',
             'L3_S33_F3865','L2_S27_F3199','L1_S24_F1632','L2_S26_F3106','L3_S29_F3333','L0_S0_F4','L0_S4_F104',
             'L0_S12_F346','L1_S24_F1581','L0_S12_F332','L3_S33_F3857',
             'Response'], dtype=np.float32 )

num_chunks_test = pd.read_csv("test_numeric.csv", 
                         usecols=['Id',
             'L3_S32_F3850','L0_S0_F0','L0_S12_F330','L1_S24_F1512','L3_S38_F3952','L3_S33_F3855',
             'L1_S24_F1514','L3_S33_F3859','L1_S24_F1118','L1_S25_F1855','L1_S24_F1723','L3_S38_F3960',
             'L0_S0_F18','L3_S30_F3494','L0_S0_F8','L0_S0_F2','L2_S27_F3129','L1_S24_F1518','L3_S30_F3749',
             'L3_S33_F3865','L2_S27_F3199','L1_S24_F1632','L2_S26_F3106','L3_S29_F3333','L0_S0_F4','L0_S4_F104',
             'L0_S12_F346','L1_S24_F1581','L0_S12_F332','L3_S33_F3857'], dtype=np.float32)


In [ ]:
#code to create graphs, the L0_S0_F0 is the most predictive var want to see it mapped against other vars
for feat in df_n_prep.columns[2:]:
    sns.factorplot(data = df_n_prep,x='L0_S0_F0',y=feat ,hue='Response')

In [ ]:
#used variation of this code to do PCA. You have to change range values depending on the size of your dataset
pca = PCA()
test_pca = pca.fit_transform(X)
pca.explained_variance_ratio_
plt.bar(range(1,24),pca.explained_variance_ratio_, alpha=0.5, align='center')
plt.step(range(1,24),np.cumsum(pca.explained_variance_ratio_), where='mid')
plt.ylabel('Explained variance ratio')
plt.xlabel('Principal components')
plt.show()

In [ ]:
#great code for breaking data into chunks and reading in from large files
num_chunks = pd.read_csv("train_numeric.csv", index_col=0,
                         usecols=list(range(969)), chunksize=100000, dtype=np.float32)
cat_chunks = pd.read_csv("train_categorical.csv", index_col=0,
                         usecols=list(range(2140)), chunksize=100000, dtype=str)

X = pd.concat([pd.concat([ nchunk, cchunk], axis=1).sample(frac=0.01)
               
               for nchunk, cchunk in zip(num_chunks,cat_chunks)])
y = pd.read_csv("train_numeric.csv", index_col=0, usecols=[0,969], dtype=np.float32).loc[X.index].values.ravel()

In [ ]:
#alt option for random samp from a dataset
# reading in training data
filename_num = 'train_numeric.csv'
#filename_date = 'train_date.csv'
filename_cat = 'train_categorical.csv'
n_num = sum(1 for lin in open('train_numeric.csv'))-1

s = 40000
chunksize_= 30000
skip = sorted(random.sample(xrange(1,n_num+1),n_num-s)) #the 0-indexed header will not be included in the skip list
df_n_prep = pd.read_csv(filename_num, skiprows=skip)

In [ ]:
#modified code to do my own feature selection via the random samp from above 
import pandas as pd
import xgboost as xgb
import operator
import random
import numpy as np
from matplotlib import pylab as plt


# note in the get data fx the first 4 lines were added to speed up process!
def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

def get_data():
    
    n_num = sum(1 for lin in open('train_categorical.csv'))-1
    s = 1000
    chunksize_= 30000
    skip = sorted(random.sample(xrange(1,n_num+1),n_num-s)) #the 0-indexed header will not be included in the skip list
    train = pd.read_csv("train_categorical.csv",skiprows=skip)
    X = train.set_index('Id')

    
    y_train = pd.read_csv("train_numeric.csv", index_col=0, usecols=[0,969], dtype=np.float32).loc[X.index]
    y_train= y_train.reset_index()
    train = pd.merge(train,y_train, how ='left',on='Id')
    features = list(train.columns[1:2141])    

    for feat in train.select_dtypes(include=['object']).columns:
        m = train.groupby([feat])['Response'].mean()
        train[feat].replace(m,inplace=True)

    x_train = train[features]

    return features, x_train, y_train


features, x_train, y_train = get_data()

y_train.drop('Id',axis=1,inplace=True)
ceate_feature_map(features)

xgb_params = {"objective": "reg:linear", "eta": 0.5, "max_depth": 4, "seed": 42, "silent": 1}
num_rounds = 500
from sklearn.preprocessing import Imputer

x_train.fillna(0,inplace=True)
dtrain = xgb.DMatrix(x_train, label=y_train)
gbdt = xgb.train(xgb_params, dtrain, num_rounds)
importance = gbdt.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')
df[:100].to_csv('important_features.csv')
#sort the top x amount to use
df.sort_values('fscore',ascending=False,inplace=True)
df[:30].to_csv('important_features.csv')

In [ ]:
#this is the normal feature importance code
import pandas as pd
import xgboost as xgb
import operator
import numpy as np
from matplotlib import pylab as plt

def ceate_feature_map(features):
    outfile = open('xgb.fmap', 'w')
    i = 0
    for feat in features:
        outfile.write('{0}\t{1}\tq\n'.format(i, feat))
        i = i + 1

    outfile.close()

def get_data():
    train = pd.read_csv("train_numeric.csv",dtype=np.float32)

    features = list(train.columns[1:969])

    y_train = train.Response

    for feat in train.select_dtypes(include=['object']).columns:
        m = train.groupby([feat])['Response'].mean()
        train[feat].replace(m,inplace=True)

    x_train = train[features]

    return features, x_train, y_train


features, x_train, y_train = get_data()
ceate_feature_map(features)

xgb_params = {"objective": "reg:linear", "eta": 0.01, "max_depth": 4, "seed": 42, "silent": 1}
num_rounds = 1000
from sklearn.preprocessing import Imputer

x_train.fillna(0,inplace=True)
dtrain = xgb.DMatrix(x_train, label=y_train)
gbdt = xgb.train(xgb_params, dtrain, num_rounds)

importance = gbdt.get_fscore(fmap='xgb.fmap')
importance = sorted(importance.items(), key=operator.itemgetter(1))

df = pd.DataFrame(importance, columns=['feature', 'fscore'])
df['fscore'] = df['fscore'] / df['fscore'].sum()

plt.figure()
df.plot()
df.plot(kind='barh', x='feature', y='fscore', legend=False, figsize=(6, 10))
plt.title('XGBoost Feature Importance')
plt.xlabel('relative importance')
plt.gcf().savefig('feature_importance_xgb.png')

In [ ]:
#here is LSTM RNN code, had to give up because it took to long to run
import theano
from theano import tensor as T
theano.config.floatX = 'float32'
from keras.models import Sequential
from keras.layers.core import Dense
from keras.layers import LSTM

t1 = test2.iloc[:,1:]
X= X.astype(theano.config.floatX)
t1 = t1.astype(theano.config.floatX)

#code to impute and normalize

#play wish shape of tensor
X = np.reshape(X,(X.shape[0],X.shape[1],1))
t2 = np.reshape(t2,(t2.shape[0],t2.shape[1],1))

model = Sequential()
model.add(LSTM(5,input_dim=X.shape[2],return_sequences=True))
model.add(LSTM(5))
model.add(Dense(1))
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])

print('fitting')
model.fit(X,y_train, nb_epoch=15, batch_size=100000,validation_split=0.1)

pred = model.predict(t2,batch_size=100000)
